<a href="https://colab.research.google.com/github/tyri0n11/distributed-system/blob/main/7_2_data_preparation_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [633 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,512 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,031 kB]
Get:13 http://secur

In [ ]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


# `StringIndexer` and `OneHotEncoder`

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext(conf=SparkConf())
spark = SparkSession(sparkContext=sc)

# Example data

In [ ]:
import pandas as pd
pdf = pd.DataFrame({
        'x1': ['a','a','b','b', 'b', 'c'],
        'x2': ['apple', 'orange', 'orange','orange', 'peach', 'peach'],
        'x3': [1, 1, 2, 2, 2, 4],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5],
        'y1': [1, 0, 1, 0, 0, 1],
        'y2': ['yes', 'no', 'no', 'yes', 'yes', 'yes']
    })
# `pdf` is pandas dataframe while `df` is Spark dataframe
df = spark.createDataFrame(pdf)
df.show()

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|orange|  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|yes|
|  b| peach|  2|2.1|  0|yes|
|  c| peach|  4|1.5|  1|yes|
+---+------+---+---+---+---+



In [ ]:
print('Type of pdf', type(pdf))
print('Type of df', type(df))

Type of pdf <class 'pandas.core.frame.DataFrame'>
Type of df <class 'pyspark.sql.dataframe.DataFrame'>


# StringIndexer

`StringIndexer` maps a string column to a index column that will be treated as a categorical column by spark. **The indices start with 0 and are ordered by label frequencies**. If it is a numerical column, the column will first be casted to a string column and then indexed by  StringIndexer.

There are three steps to implement the StringIndexer

1. Build the StringIndexer model: specify the input column and output column names.
2. Learn the StringIndexer model: fit the model with your data.
3. Execute the indexing: call the transform function to execute the indexing process.

### Example: `StringIndex` column "x1"

In [ ]:
from pyspark.ml.feature import StringIndexer

# build indexer
string_indexer = StringIndexer(inputCol='x1', outputCol='indexed_x1')

# learn the model
string_indexer_model = string_indexer.fit(df)

# transform the data
df_stringindexer = string_indexer_model.transform(df)

# resulting df
df_stringindexer.show()

+---+------+---+---+---+---+----------+
| x1|    x2| x3| x4| y1| y2|indexed_x1|
+---+------+---+---+---+---+----------+
|  a| apple|  1|2.4|  1|yes|       1.0|
|  a|orange|  1|2.5|  0| no|       1.0|
|  b|orange|  2|3.5|  1| no|       0.0|
|  b|orange|  2|1.4|  0|yes|       0.0|
|  b| peach|  2|2.1|  0|yes|       0.0|
|  c| peach|  4|1.5|  1|yes|       2.0|
+---+------+---+---+---+---+----------+



### Your task `StringIndex` column "x2"

In [ ]:
from pyspark.ml.feature import StringIndexer
# build indexer
string_indexer_2 = StringIndexer(inputCol='x2', outputCol='indexed_x2')
# learn the model
string_indexer_model_2 = string_indexer_2.fit(df)
# transform the data
df_stringindexer = string_indexer_model_2.transform(df)
# resulting df
df_stringindexer.show()

+---+------+---+---+---+---+----------+
| x1|    x2| x3| x4| y1| y2|indexed_x2|
+---+------+---+---+---+---+----------+
|  a| apple|  1|2.4|  1|yes|       2.0|
|  a|orange|  1|2.5|  0| no|       0.0|
|  b|orange|  2|3.5|  1| no|       0.0|
|  b|orange|  2|1.4|  0|yes|       0.0|
|  b| peach|  2|2.1|  0|yes|       1.0|
|  c| peach|  4|1.5|  1|yes|       1.0|
+---+------+---+---+---+---+----------+



From the result above, we can see that (a, b, c) in column x1 are converted to (1.0, 0.0, 2.0). They are ordered by their frequencies in column x1.



# OneHotEncoder

**`OneHotEncoder`** converts each categories of a **StringIndexed** column to a `sparse vector`. Each sparse vector has **at most one single active elements** that indicate the category index.

In [ ]:
df.show(4)

+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|orange|  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|yes|
+---+------+---+---+---+---+
only showing top 4 rows



In [ ]:
df_ohe = df.select('x1')
df_ohe.show()

+---+
| x1|
+---+
|  a|
|  a|
|  b|
|  b|
|  b|
|  c|
+---+



### `StringIndex` column 'x1'

In [ ]:
df_x1_indexed = StringIndexer(inputCol='x1', outputCol='indexed_x1').fit(df_ohe).transform(df_ohe)
df_x1_indexed.show()

+---+----------+
| x1|indexed_x1|
+---+----------+
|  a|       1.0|
|  a|       1.0|
|  b|       0.0|
|  b|       0.0|
|  b|       0.0|
|  c|       2.0|
+---+----------+



'x1' has three categories: 'a', 'b' and 'c',  which corresponding string indices 1.0, 0.0 and 2.0, respectively.

### Mapping string indices to sparse vectors

* Encoding format: 'string index': ['string indices vector size', 'index of string index in string indices vector', **1.0** ]

Here the string indices vector is `[0.0, 1.0, 2.0]`. Therefore, the mapping between string indices and sparse vectors are:
* `0.0: [3, [0], [1.0]]`
* `1.0: [3, [1], [1.0]]`
* `2.0: [3, [2], [1.0]]`

After we convert all sparse vectors to dense vectors, we get:

In [ ]:
from pyspark.ml.linalg import DenseVector, SparseVector, DenseMatrix, SparseMatrix
x = [SparseVector(3, {0: 1.0}).toArray()] + \
    [SparseVector(3, {1: 1.0}).toArray()] + \
    [SparseVector(3, {2: 1.0}).toArray()]

import numpy as np
np.array(x)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

**The obtained matrix is exactly the matrix that we would use to represent our categorical variable in a statistical class**.

### One more step to go

`OneHotEncoder` by default will drop the last category. So the **string indices vector** becomes `[0.0, 1.0]`, and the mappings between string indices and sparse vectors are:

* `0.0: [2, [0], [1.0]]`
* `1.0: [2, [1], [1.0]]`
* `2.0: [2, [], []]`

We use a sparse vector that has **no active element**(basically all elements are 0's) to represent the last category.

# Verify

### OneHotEncode column 'indexed_x1'

In [ ]:
from pyspark.ml.feature import OneHotEncoder

In [ ]:
# review `df_x1_indexed`, what is it?
df_x1_indexed.show(5)

+---+----------+
| x1|indexed_x1|
+---+----------+
|  a|       1.0|
|  a|       1.0|
|  b|       0.0|
|  b|       0.0|
|  b|       0.0|
+---+----------+
only showing top 5 rows



In [ ]:
OneHotEncoder(inputCol='indexed_x1', outputCol='encoded_x1').fit(df_x1_indexed).transform(df_x1_indexed).show()

+---+----------+-------------+
| x1|indexed_x1|   encoded_x1|
+---+----------+-------------+
|  a|       1.0|(2,[1],[1.0])|
|  a|       1.0|(2,[1],[1.0])|
|  b|       0.0|(2,[0],[1.0])|
|  b|       0.0|(2,[0],[1.0])|
|  b|       0.0|(2,[0],[1.0])|
|  c|       2.0|    (2,[],[])|
+---+----------+-------------+



### Specify to not drop the last category

If we choose to not drop the last category, we get the expected results.

In [ ]:
OneHotEncoder(dropLast=False, inputCol='indexed_x1', outputCol='encoded_x1').fit(df_x1_indexed).transform(df_x1_indexed).show()

+---+----------+-------------+
| x1|indexed_x1|   encoded_x1|
+---+----------+-------------+
|  a|       1.0|(3,[1],[1.0])|
|  a|       1.0|(3,[1],[1.0])|
|  b|       0.0|(3,[0],[1.0])|
|  b|       0.0|(3,[0],[1.0])|
|  b|       0.0|(3,[0],[1.0])|
|  c|       2.0|(3,[2],[1.0])|
+---+----------+-------------+



## Exercise:
**Do the same OneHotEncoder for the columns `x2` and `y2`**

# Vector assembler

## Example data

In [ ]:
import pandas as pd
pdf = pd.DataFrame({
        'x1': ['a','a','b','b', 'b', 'c'],
        'x2': ['apple', 'orange', 'orange','orange', 'peach', 'peach'],
        'x3': [1, 1, 2, 2, 2, 4],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5],
        'y1': [1, 0, 1, 0, 0, 1],
        'y2': ['yes', 'no', 'no', 'yes', 'yes', 'yes']
    })
df = spark.createDataFrame(pdf)
df.show()

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|orange|  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|yes|
|  b| peach|  2|2.1|  0|yes|
|  c| peach|  4|1.5|  1|yes|
+---+------+---+---+---+---+



# VectorAssembler

To fit a ML model in pyspark, we need to combine all feature columns into one single column of vectors: the **featuresCol**. The `VectorAssembler` can be used to combine multiple **`OneHotEncoder` columns** and **other continuous variable columns** into one single column.

The example below shows how to combine three OneHotEncoder columns and one numeric column into a **featureCol** column.



## StringIndex and OneHotEncode categorical columns

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

In [ ]:
all_stages = [StringIndexer(inputCol=c, outputCol='idx_' + c) for c in ['x1', 'x2', 'x3']] + \
             [OneHotEncoder(inputCol='idx_' + c, outputCol='ohe_' + c) for c in ['x1', 'x2', 'x3']]
all_stages

[StringIndexer_20376647e69c,
 StringIndexer_3ad93263c2ec,
 StringIndexer_e31d2f557eb3,
 OneHotEncoder_5bfbbfd49e10,
 OneHotEncoder_fccfaa37006a,
 OneHotEncoder_00432b4231d4]

In [ ]:
df_new = Pipeline(stages=all_stages).fit(df).transform(df)
df_new.show()

+---+------+---+---+---+---+------+------+------+-------------+-------------+-------------+
| x1|    x2| x3| x4| y1| y2|idx_x1|idx_x2|idx_x3|       ohe_x1|       ohe_x2|       ohe_x3|
+---+------+---+---+---+---+------+------+------+-------------+-------------+-------------+
|  a| apple|  1|2.4|  1|yes|   1.0|   2.0|   1.0|(2,[1],[1.0])|    (2,[],[])|(2,[1],[1.0])|
|  a|orange|  1|2.5|  0| no|   1.0|   0.0|   1.0|(2,[1],[1.0])|(2,[0],[1.0])|(2,[1],[1.0])|
|  b|orange|  2|3.5|  1| no|   0.0|   0.0|   0.0|(2,[0],[1.0])|(2,[0],[1.0])|(2,[0],[1.0])|
|  b|orange|  2|1.4|  0|yes|   0.0|   0.0|   0.0|(2,[0],[1.0])|(2,[0],[1.0])|(2,[0],[1.0])|
|  b| peach|  2|2.1|  0|yes|   0.0|   1.0|   0.0|(2,[0],[1.0])|(2,[1],[1.0])|(2,[0],[1.0])|
|  c| peach|  4|1.5|  1|yes|   2.0|   1.0|   2.0|    (2,[],[])|(2,[1],[1.0])|    (2,[],[])|
+---+------+---+---+---+---+------+------+------+-------------+-------------+-------------+



## Assemble feature columns into one single **feacturesCol** with **`VectorAssembler`**

In [ ]:
df_assembled = VectorAssembler(inputCols=['ohe_x1', 'ohe_x2', 'ohe_x3', 'x4'], outputCol='featuresCol')\
    .transform(df_new)\
    .drop('idx_x1', 'idx_x2', 'idx_x3')
df_assembled.show(truncate=False)

+---+------+---+---+---+---+-------------+-------------+-------------+-----------------------------+
|x1 |x2    |x3 |x4 |y1 |y2 |ohe_x1       |ohe_x2       |ohe_x3       |featuresCol                  |
+---+------+---+---+---+---+-------------+-------------+-------------+-----------------------------+
|a  |apple |1  |2.4|1  |yes|(2,[1],[1.0])|(2,[],[])    |(2,[1],[1.0])|(7,[1,5,6],[1.0,1.0,2.4])    |
|a  |orange|1  |2.5|0  |no |(2,[1],[1.0])|(2,[0],[1.0])|(2,[1],[1.0])|[0.0,1.0,1.0,0.0,0.0,1.0,2.5]|
|b  |orange|2  |3.5|1  |no |(2,[0],[1.0])|(2,[0],[1.0])|(2,[0],[1.0])|[1.0,0.0,1.0,0.0,1.0,0.0,3.5]|
|b  |orange|2  |1.4|0  |yes|(2,[0],[1.0])|(2,[0],[1.0])|(2,[0],[1.0])|[1.0,0.0,1.0,0.0,1.0,0.0,1.4]|
|b  |peach |2  |2.1|0  |yes|(2,[0],[1.0])|(2,[1],[1.0])|(2,[0],[1.0])|[1.0,0.0,0.0,1.0,1.0,0.0,2.1]|
|c  |peach |4  |1.5|1  |yes|(2,[],[])    |(2,[1],[1.0])|(2,[],[])    |(7,[3,6],[1.0,1.5])          |
+---+------+---+---+---+---+-------------+-------------+-------------+---------------------

## Convert sparse vectors in featuresCol to dense vectors

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.linalg import SparseVector, DenseVector

In [ ]:
def dense_features_col(x):
    return(x.toArray().dtype)
dense_features_col_udf = udf(dense_features_col, returnType=StringType())

In [ ]:
df_assembled.rdd.map(lambda x: x['featuresCol']).take(6)

[SparseVector(7, {1: 1.0, 5: 1.0, 6: 2.4}),
 DenseVector([0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.5]),
 DenseVector([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 3.5]),
 DenseVector([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.4]),
 DenseVector([1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 2.1]),
 SparseVector(7, {3: 1.0, 6: 1.5})]

In [ ]:
df_assembled.rdd.map(lambda x: list(x['featuresCol'].toArray())).take(6)

[[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 2.4],
 [0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.5],
 [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 3.5],
 [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.4],
 [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 2.1],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.5]]

## Practice

In [ ]:
import pandas as pd
pdf2 = pd.DataFrame({
        'x1': ['a','a','b','b', 'b', 'c'],
        'x2': ['apple', 'orange', 'orange','orange', 'peach', 'peach'],
        'x3': [1, 1, 2, 2, 2, 4],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5],
        'x5': ['man', 'woman', 'man', 'man', 'man', 'woman'],
        'x6': [10.3, 11.4, 45.3, 32.5, 13.8, 17.2],
        'x7': ['911', '113', '115', '113', '911', '115'],
        'y1': [1, 0, 1, 0, 0, 1],
        'y2': ['yes', 'no', 'no', 'yes', 'yes', 'yes']
    })
df2 = spark.createDataFrame(pdf2)
df2.show()

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+------+---+---+-----+----+---+---+---+
| x1|    x2| x3| x4|   x5|  x6| x7| y1| y2|
+---+------+---+---+-----+----+---+---+---+
|  a| apple|  1|2.4|  man|10.3|911|  1|yes|
|  a|orange|  1|2.5|woman|11.4|113|  0| no|
|  b|orange|  2|3.5|  man|45.3|115|  1| no|
|  b|orange|  2|1.4|  man|32.5|113|  0|yes|
|  b| peach|  2|2.1|  man|13.8|911|  0|yes|
|  c| peach|  4|1.5|woman|17.2|115|  1|yes|
+---+------+---+---+-----+----+---+---+---+



#### Your task: Please do the Assemble feature columns (of all categorical and numerical features) into one single **feacturesCol** with **`VectorAssembler`**
**Hint: Categorical features (`x1`, `x2`, `x3`, `x5`, `x7`) and numerical features (`x4`, `x6`)**

#### Exercise: Do the Assemble feature columns (of all categorical and numerical features) into one single **feacturesCol** with **`VectorAssembler`** for **`mtcars`** dataset

#### Exercise: Do the Assemble feature columns (of all categorical and numerical features) into one single **feacturesCol** with **`VectorAssembler`** for **`titanic`** dataset